In [29]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pd_read
import functools 

SYM = "Symbol"
QTY = "Qty Available"
WEI = "Weight"
MEAN = "Mean"
STD = "Std"
VARI = "Variance"



In [30]:
#read portfolio from actual zerodha sheet

PORTFOLIO_LOC = '../data/portfolio.xlsx' # replace the file in the data directory with you portfolio with name as portfolio.xlsx
portfolio_file = open(PORTFOLIO_LOC, 'rb')
portfolio = pd.read_excel(io = portfolio_file, usecols = "B,E", skiprows = 9, dtype =  {'b': np.str, 'e': np.int32} )

In [31]:
#ZERODHA sheet corrections 

#IDK WTF is AXISBANK6 in ZERODHA 
portfolio[SYM] = portfolio[SYM].replace(to_replace = "AXISBANK6", value = "AXISBANK")
#yahooo expects BS/NS suffix to symbol , assuming al stock in NSE
portfolio[SYM] += ".NS"

In [32]:
# make a weighted portfolio 
total_qty = portfolio[QTY].sum()
portfolio[WEI] = portfolio[QTY] / total_qty

In [33]:
# add mean std var for each stock to portfolio
stock_data = pd.DataFrame()
for stock in portfolio[SYM]:
    stock_data[stock]= pd_read.DataReader(stock, data_source='yahoo', start='2020-1-1')['Close']
stock_returns = np.log(stock_data/stock_data.shift(1))
portfolio = portfolio.merge(pd.DataFrame().join([stock_returns.mean().to_frame(MEAN),stock_returns.std().to_frame(STD),stock_returns.std().to_frame(VARI)],how = 'outer'),left_index = False, right_index = True, how = 'left', left_on = SYM) 
portfolio

,Symbol,Qty Available,Weight,Mean,Std,Variance
0,BHEL.NS,8,0.084211,-0.001912,0.039817,0.039817
1,HUDCO.NS,4,0.042105,-0.000708,0.033307,0.033307
2,LUPIN.NS,1,0.010526,0.001089,0.026045,0.026045
3,DABUR.NS,1,0.010526,0.000541,0.021097,0.021097
4,BIOCON.NS,4,0.042105,0.001531,0.025685,0.025685
5,NLCINDIA.NS,1,0.010526,-0.000713,0.027458,0.027458
6,LICHSGFIN.NS,1,0.010526,-0.001804,0.040060,0.040060
7,ITC.NS,4,0.042105,-0.001693,0.026963,0.026963
8,UPL.NS,1,0.010526,-0.001498,0.035066,0.035066
9,ICICIBANK.NS,1,0.010526,-0.001387,0.036095,0.036095


In [34]:
# calculate covariance matrix 
cov_matrix = stock_returns.cov()

In [35]:
# portfolio risk
portfolio_risk = np.dot(portfolio[WEI].T, np.dot(cov_matrix, portfolio[WEI]))
portfolio_risk

0.0005535400150538315

In [36]:
#portfolio_volatility 
portfolio_volatility = portfolio_risk ** 0.5
portfolio_volatility

0.023527431118884005

In [37]:
#diversifiable risk  = portfolio risk - sum of  weigth **2 - variance for each stock 
diversifiabe_risk = portfolio_risk - (portfolio[WEI]**2 * portfolio[VARI]).sum()
diversifiabe_risk

-0.003153256285841866

In [38]:
#%return for portfolio since 1st Jan 2020 (assuming constant portfolio)
np.dot(stock_returns.mean(), portfolio[WEI]) * 100

-0.06353870807996014

In [41]:
#highest reurn
portfolio.iloc[portfolio[MEAN].idxmax()]

Symbol            BIOCON.NS
Qty Available             4
Weight            0.0421053
Mean             0.00153123
Std               0.0256853
Variance          0.0256853
Name: 4, dtype: object

In [42]:
#lowest return
portfolio.iloc[portfolio[MEAN].idxmin()]

Symbol             SBIN.NS
Qty Available           11
Weight            0.115789
Mean             -0.002602
Std              0.0317397
Variance         0.0317397
Name: 17, dtype: object